### Random Forest Classifier

In [140]:
import pandas as pd
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test = pd.read_csv("../data/processed/X_test.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")

In [141]:
from sklearn.ensemble import RandomForestClassifier

X_id = X_test["PassengerId"]
y_id = y_test["PassengerId"]
X_train = X_train.drop(columns=["Unnamed: 0","PassengerId"])
X_test = X_test.drop(columns=["Unnamed: 0","PassengerId"])
y_train = y_train.drop(columns=["Unnamed: 0","PassengerId"])
y_test = y_test.drop(columns=["Unnamed: 0","PassengerId"])
import numpy as np
y_test = np.array(y_test["Survived"])
y_train = np.array(y_train["Survived"])

rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [142]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [143]:
rf.score(X_test, y_test)

0.7798507462686567

In [144]:
from sklearn.metrics import precision_score
precision_score(y_test, rf.predict(X_test))

0.7954545454545454

In [145]:
a = rf.predict(X_test)

In [146]:
b = pd.DataFrame(a, columns=["Survived"])
b.mean()

Survived    0.328358
dtype: float64

In [147]:
final = pd.DataFrame(y_id)
final["Survived"] = pd.DataFrame(a)

In [148]:
final

,PassengerId,Survived
0,710,0
1,440,0
2,841,0
3,721,1
4,40,1
...,...,...
263,822,0
264,634,0
265,457,0
266,501,0


In [149]:
validate = pd.read_csv("../data/raw/validate.csv")

In [150]:
def clean_titanic(df):
    df = df.drop(
        columns=[
            "Name",
            "Ticket",
            "Cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)

    return df

In [151]:
validate = clean_titanic(validate)

In [152]:
import pandas as pd
from sklearn import model_selection, impute, preprocessing
from sklearn.experimental import enable_iterative_imputer
def process(df):
    
    num_cols = [
        "Age",
        "Fare",
    ]

    imputer = impute.IterativeImputer()
    df.loc[:,num_cols] = imputer.fit_transform(df[num_cols])


    cols = "Pclass,Age,SibSp,Parch,Fare".split(",")
    sca = preprocessing.StandardScaler()

    df.loc[:,cols] = sca.fit_transform(df[cols])

    return df

In [153]:
validate = process(validate)

In [154]:
final_1 = pd.DataFrame(validate["PassengerId"])
validate = validate.drop(columns=["PassengerId"])


In [155]:
final_2 = rf.predict(validate)

In [156]:
final_1["Survived"] = pd.DataFrame(final_2)

In [157]:
from src import src_path
from os import path

In [158]:
src_path

'/home/gbpagano/ihm/titanic/notebooks'

In [159]:
final_1.to_csv(path.join(src_path,"random_forest.csv"), header=True)

In [161]:
sub = pd.Series(final_2, index=final_1["PassengerId"], name="Survived")

In [164]:
sub.to_csv(path.join(src_path,"random_forest2.csv"), header=True)
